<a href="https://colab.research.google.com/github/tung134245/Recomendation_Amazon_BigData/blob/main/Beauty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
!pip install pyspark==3.1.1

In [4]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row, SQLContext

In [5]:
sc = SparkContext()

In [6]:
spark = SparkSession(sc)

In [7]:
sqlContext = SQLContext(sc)

In [8]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [9]:
%cd '/content/gdrive/My Drive/BigData/Chapter9'

/content/gdrive/My Drive/BigData/Chapter9


In [10]:
from pyspark.sql.functions import *


In [11]:
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types
from pyspark.sql.functions import col, udf
from datetime import datetime
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month, dayofmonth, weekofyear
from pyspark.ml.feature import *

## Đọc dữ liệu

In [12]:
df = spark.read.json('/content/gdrive/My Drive/BigData/Chapter9/Beauty_5.json')
df.show()

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|7806397051|  [3, 4]|    1.0|Very oily and cre...|01 30, 2014|A1YJEY40YUW4SE|              Andrea|Don't waste your ...|    1391040000|
|7806397051|  [1, 1]|    3.0|This palette was ...|04 18, 2014| A60XNB876KYML|          Jessica H.|         OK Palette!|    1397779200|
|7806397051|  [0, 1]|    4.0|The texture of th...| 09 6, 2013|A3G6XNM240RMWA|               Karen|       great quality|    1378425600|
|7806397051|  [2, 2]|    2.0|I really can't te...| 12 8, 2013|A1PQFP6SAJ6D80|               Norah|Do not work on my...|    1386460800|
|7806397051|  [0, 0]|    3.0|It was a little s...|10 19

In [13]:
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



## Trích xuất dữ liệu

In [15]:
df_sub = df.select(['asin', 'overall', 'reviewerID'])

In [16]:
df_sub.count()

198502

In [17]:
df_sub.describe().show()

+-------+-------------------+------------------+--------------------+
|summary|               asin|           overall|          reviewerID|
+-------+-------------------+------------------+--------------------+
|  count|             198502|            198502|              198502|
|   mean|9.326864459742857E9| 4.190391028805755|                null|
| stddev| 8.39829661439545E8|1.1665795141887418|                null|
|    min|         7806397051|               1.0|A00414041RD0BXM6W...|
|    max|         B00LLPT4HI|               5.0|       AZZZLM1E5JJ8C|
+-------+-------------------+------------------+--------------------+



In [18]:
df_sub.printSchema()

root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewerID: string (nullable = true)



## Kiểm tra độ dày của ma trận

In [19]:
users = df_sub.select("reviewerID").distinct().count()
products = df_sub.select("asin").distinct().count()
numerator = df_sub.count()

In [20]:
display(numerator, users, products)

198502

22363

12101

In [21]:
denominator = users * products
denominator

270614663

In [22]:
sparsity = 1 - (numerator*1.0 / denominator)
sparsity

0.9992664772935825

In [23]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

## Tiền xử lý

In [24]:
from pyspark.ml.feature import *

In [25]:
indexer = StringIndexer(inputCol='asin',
                        outputCol='asin_idx')

indexer_model = indexer.fit(df_sub)

df_indexed = indexer_model.transform(df_sub)

indexer_ = StringIndexer(inputCol='reviewerID',
                         outputCol='reviewerID_idx')
indexer_model_ = indexer_.fit(df_indexed)
df_indexed = indexer_model_.transform(df_indexed)

In [26]:
df_indexed.show(5, False)

+----------+-------+--------------+--------+--------------+
|asin      |overall|reviewerID    |asin_idx|reviewerID_idx|
+----------+-------+--------------+--------+--------------+
|7806397051|1.0    |A1YJEY40YUW4SE|6194.0  |16983.0       |
|7806397051|3.0    |A60XNB876KYML |6194.0  |10399.0       |
|7806397051|4.0    |A3G6XNM240RMWA|6194.0  |5985.0        |
|7806397051|2.0    |A1PQFP6SAJ6D80|6194.0  |11765.0       |
|7806397051|3.0    |A38FVHZTNQ271F|6194.0  |5910.0        |
+----------+-------+--------------+--------+--------------+
only showing top 5 rows



In [27]:
df_indexed.describe().show()

+-------+-------------------+------------------+--------------------+------------------+-----------------+
|summary|               asin|           overall|          reviewerID|          asin_idx|   reviewerID_idx|
+-------+-------------------+------------------+--------------------+------------------+-----------------+
|  count|             198502|            198502|              198502|            198502|           198502|
|   mean|9.326864459742857E9| 4.190391028805755|                null| 3033.162703650341|7710.090784979497|
| stddev| 8.39829661439545E8|1.1665795141887418|                null|3236.3522542646283|6674.790355275204|
|    min|         7806397051|               1.0|A00414041RD0BXM6W...|               0.0|              0.0|
|    max|         B00LLPT4HI|               5.0|       AZZZLM1E5JJ8C|           12100.0|          22362.0|
+-------+-------------------+------------------+--------------------+------------------+-----------------+



In [28]:
train_data, test_data = df_indexed.randomSplit([0.8, 0.2])

## Xây dựng mô hình

In [29]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [30]:
als = ALS(maxIter=20,
          regParam=0.01,
          rank = 25,
          userCol="reviewerID_idx",
          itemCol="asin_idx",
          ratingCol="overall",
          coldStartStrategy="drop",
          nonnegative=True)
model = als.fit(train_data)

In [31]:
predictions = model.transform(test_data)

In [32]:
evaluator = RegressionEvaluator(metricName="rmse",
                                labelCol="overall",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [33]:
print("RMSE:", rmse)

RMSE: 1.44432173922828


## Tiến hành đề xuất

In [36]:
user_recs = model.recommendForAllUsers(5)

In [37]:
user_recs.count()

22360

In [38]:
user_recs.show(10, truncate=False)

+--------------+-------------------------------------------------------------------------------------------------+
|reviewerID_idx|recommendations                                                                                  |
+--------------+-------------------------------------------------------------------------------------------------+
|1580          |[{8541, 6.7651796}, {11011, 6.5288467}, {11437, 6.489747}, {10415, 6.3831987}, {3604, 6.2851367}]|
|4900          |[{10838, 7.509685}, {5137, 7.397314}, {7372, 7.3265157}, {4452, 7.240062}, {5606, 7.2139454}]    |
|5300          |[{3702, 6.548118}, {6069, 6.476222}, {10367, 6.4254704}, {2692, 6.4173827}, {10340, 6.3819003}]  |
|6620          |[{2730, 6.206671}, {4934, 5.847941}, {6240, 5.8326526}, {9641, 5.8168974}, {10048, 5.756711}]    |
|7240          |[{8421, 6.082803}, {4923, 6.060949}, {8358, 5.9993167}, {5115, 5.9568667}, {9142, 5.942891}]     |
|7340          |[{5115, 6.255713}, {4327, 5.604293}, {6795, 5.5822797}, {11653, 

In [39]:
df_reviewer_reviewer_id = df_indexed.select('reviewerID_idx', 'reviewerID').distinct()
df_reviewer_reviewer_id.show(5, False)

+--------------+--------------+
|reviewerID_idx|reviewerID    |
+--------------+--------------+
|7501.0        |A339O8ZW72WHZ0|
|2.0           |AKMEY1BSHSDG7 |
|9739.0        |A30B9UTVDTUQ7Y|
|11415.0       |A1FMYCX030FIDE|
|718.0         |AUFB3GQJV10P0 |
+--------------+--------------+
only showing top 5 rows



In [41]:
df_asin_asin_idx = df_indexed.select('asin_idx', 'asin').distinct()
df_asin_asin_idx.show(5, False)

+--------+----------+
|asin_idx|asin      |
+--------+----------+
|6203.0  |B00005TZU8|
|9941.0  |B00005UN90|
|3659.0  |B000142ZFS|
|810.0   |B00027DDOQ|
|3399.0  |B0006IJA5C|
+--------+----------+
only showing top 5 rows



In [42]:
new_user_recs = user_recs.join(df_reviewer_reviewer_id, on=['reviewerID_idx'], how='left')
new_user_recs.show(10, False)

+--------------+------------------------------------------------------------------------------------------------+--------------+
|reviewerID_idx|recommendations                                                                                 |reviewerID    |
+--------------+------------------------------------------------------------------------------------------------+--------------+
|299           |[{10641, 7.1514482}, {2767, 6.8824263}, {10447, 6.8379774}, {5115, 6.835708}, {4740, 6.769886}] |A2CZPM110DW516|
|305           |[{2086, 8.205831}, {815, 7.8249636}, {2623, 7.601781}, {130, 7.5908756}, {8588, 7.5855093}]     |A3EPHBMU07LZ50|
|496           |[{4386, 5.972968}, {7914, 5.8665743}, {11789, 5.8403316}, {6687, 5.830493}, {4502, 5.792334}]   |A2A5C9IQ06CG9N|
|558           |[{9125, 7.2278113}, {3523, 7.0216823}, {3162, 6.7413983}, {8617, 6.6723247}, {5237, 6.614396}]  |A9LWDP1HECAU0 |
|596           |[{8701, 6.0224476}, {4327, 6.005945}, {3701, 5.9337296}, {7275, 5.906786}, {4538,

In [46]:
revi_Id = 'AJK5XGCM6M68A'
result = new_user_recs.filter(new_user_recs['reviewerID'] == revi_Id)
result.show(truncate=False)

+--------------+--------------------------------------------------------------------------------------------+-------------+
|reviewerID_idx|recommendations                                                                             |reviewerID   |
+--------------+--------------------------------------------------------------------------------------------+-------------+
|21521         |[{5115, 6.334028}, {9900, 6.0763755}, {8940, 5.927455}, {2129, 5.9069757}, {9199, 5.904381}]|AJK5XGCM6M68A|
+--------------+--------------------------------------------------------------------------------------------+-------------+



In [48]:
result = result.select(result.reviewerID, explode(result.recommendations))
result.show(4)

+-------------+-----------------+
|   reviewerID|              col|
+-------------+-----------------+
|AJK5XGCM6M68A| {5115, 6.334028}|
|AJK5XGCM6M68A|{9900, 6.0763755}|
|AJK5XGCM6M68A| {8940, 5.927455}|
|AJK5XGCM6M68A|{2129, 5.9069757}|
+-------------+-----------------+
only showing top 4 rows



In [49]:
result.printSchema()

root
 |-- reviewerID: string (nullable = true)
 |-- col: struct (nullable = true)
 |    |-- asin_idx: integer (nullable = true)
 |    |-- rating: float (nullable = true)



In [50]:
result = result.withColumn('asin_idx', result.col.getField('asin_idx'))
result = result.withColumn('rating', result.col.getField('rating'))
result.show(5)

+-------------+-----------------+--------+---------+
|   reviewerID|              col|asin_idx|   rating|
+-------------+-----------------+--------+---------+
|AJK5XGCM6M68A| {5115, 6.334028}|    5115| 6.334028|
|AJK5XGCM6M68A|{9900, 6.0763755}|    9900|6.0763755|
|AJK5XGCM6M68A| {8940, 5.927455}|    8940| 5.927455|
|AJK5XGCM6M68A|{2129, 5.9069757}|    2129|5.9069757|
|AJK5XGCM6M68A| {9199, 5.904381}|    9199| 5.904381|
+-------------+-----------------+--------+---------+



In [51]:
recs = result.join(df_asin_asin_idx, on=['asin_idx'], how='left')
recs.show(10, False)

+--------+-------------+-----------------+---------+----------+
|asin_idx|reviewerID   |col              |rating   |asin      |
+--------+-------------+-----------------+---------+----------+
|2129    |AJK5XGCM6M68A|{2129, 5.9069757}|5.9069757|B00ELN94GS|
|8940    |AJK5XGCM6M68A|{8940, 5.927455} |5.927455 |B0029OIWNC|
|5115    |AJK5XGCM6M68A|{5115, 6.334028} |6.334028 |B001H3JQ0E|
|9900    |AJK5XGCM6M68A|{9900, 6.0763755}|6.0763755|B00H8JPMX6|
|9199    |AJK5XGCM6M68A|{9199, 5.904381} |5.904381 |B00429S19C|
+--------+-------------+-----------------+---------+----------+

